In [1]:


'''
Los algoritmos de creación de árboles están en el submódulo de sklearn.tree

En cuanto al tipo de algoritmo para crear árboles, scikit-learn usa una versión
optimizada del algoritmo CART (Classification and Regression Trees), que permite 
usar árboles de decisión tanto para problemas de clasificación como de regresión.
'''

import os
import pandas as pd
import numpy as np





#%%
url = "https://raw.githubusercontent.com/JuanCarlosCS19/ML_Toulouse_ENE_2024/main/Clase2/titanic.csv"
mi_data = pd.read_csv(url)
mi_data.head()

,superviviente,clase_billete,genero,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,puerto_salida
0,0,3,hombre,22.0,1,0,7.2500,S
1,1,1,mujer,38.0,1,0,71.2833,C
2,1,3,mujer,26.0,0,0,7.9250,S
3,1,1,mujer,35.0,1,0,53.1000,S
4,0,3,hombre,35.0,0,0,8.0500,S


In [2]:

#%%

from sklearn import tree
from sklearn.model_selection import cross_val_score

#%%

columnas_categoricas = ["genero", "puerto_salida"]
datos_categoricos = pd.get_dummies(mi_data[columnas_categoricas])




In [3]:
#%%
pasajeros = (
    pd.concat([
        mi_data.drop(columnas_categoricas, axis=1),
        datos_categoricos
    ],axis=1
    )
)

#%%
pasajeros.edad = pasajeros.edad.fillna(pasajeros.edad.mean())
pasajeros

,superviviente,clase_billete,edad,n_hermanos_esposos,n_hijos_padres,precio_billete,genero_hombre,genero_mujer,puerto_salida_C,puerto_salida_Q,puerto_salida_S
0,0,3,22.000000,1,0,7.2500,1,0,0,0,1
1,1,1,38.000000,1,0,71.2833,0,1,1,0,0
2,1,3,26.000000,0,0,7.9250,0,1,0,0,1
3,1,1,35.000000,1,0,53.1000,0,1,0,0,1
4,0,3,35.000000,0,0,8.0500,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.0000,1,0,0,0,1
887,1,1,19.000000,0,0,30.0000,0,1,0,0,1
888,0,3,29.699118,1,2,23.4500,0,1,0,0,1
889,1,1,26.000000,0,0,30.0000,1,0,1,0,0


In [4]:
pasajeros.columns

Index(['superviviente', 'clase_billete', 'edad', 'n_hermanos_esposos',
       'n_hijos_padres', 'precio_billete', 'genero_hombre', 'genero_mujer',
       'puerto_salida_C', 'puerto_salida_Q', 'puerto_salida_S'],
      dtype='object')

In [5]:
#%%
import graphviz

variable_objetivo = pasajeros["superviviente"]
variables_independientes = pasajeros[['clase_billete', 'edad', 'n_hermanos_esposos',
       'n_hijos_padres', 'precio_billete', 'genero_hombre', 'genero_mujer',
       'puerto_salida_C', 'puerto_salida_Q', 'puerto_salida_S']]


#%%
arbol = tree.DecisionTreeClassifier() 
arbol.fit(variables_independientes, variable_objetivo)

#%%
cross_val_score(arbol,
                variables_independientes, 
                variable_objetivo, 
                scoring="roc_auc", 
                cv=40).mean()






0.7822191697191696

In [ ]:
#%%

os.environ["PATH"] = os.pathsep + "C:\\Program Files\\Graphviz\\bin"

def dibujar_arbol(arbol):
    dot_data = tree.export_graphviz(arbol, out_file=None,
                                    feature_names=pasajeros.drop("superviviente",axis=1).columns,
                                    class_names = pasajeros.superviviente.astype(str),
                                    filled=True # Reemplaza con tu ruta exacta
                                    )
    graph = graphviz.Source(dot_data)
    graph.format = "png"
    graph.render("arbol", view=True)

dibujar_arbol(arbol)


In [16]:
import shutil

def find_graphviz_executable():
    # Buscar el ejecutable 'dot' en el PATH
    graphviz_executable = shutil.which('dot')
    
    if graphviz_executable:
        print(f"Graphviz está instalado en: {graphviz_executable}")
    else:
        print("Graphviz no está instalado o no se encuentra en el PATH.")

find_graphviz_executable()

# https://graphviz.org/download/

Graphviz está instalado en: C:\Program Files\Graphviz\bin\dot.EXE


In [25]:
#%%

'''
Se puede exportar el árbol y abrirlo posteriormente con graphviz desde la terminal
(o desde la página http://webgraphviz.com/ que renderiza archivos de graphviz)
'''

tree.export_graphviz(arbol, out_file="arbol.dot",
                     feature_names=pasajeros.drop("superviviente",axis=1).columns,
                     class_names = pasajeros.superviviente.astype(str),
                     filled=True)


## webgraphviz.com



In [26]:
#%%

'''Podemos ver también cuáles son las variables más importantes'''

dict(zip(
    pasajeros.drop("superviviente", axis=1),
    arbol.feature_importances_
))


sorted(zip(
    arbol.feature_importances_,
    pasajeros.drop("superviviente", axis=1)), 
    reverse = True)



[(0.30933518862833864, 'genero_mujer'),
 (0.25056951628416063, 'edad'),
 (0.23194335616269016, 'precio_billete'),
 (0.1094608741944015, 'clase_billete'),
 (0.049830435730651476, 'n_hermanos_esposos'),
 (0.028825789772450237, 'n_hijos_padres'),
 (0.009770634925298833, 'puerto_salida_S'),
 (0.005645800696061534, 'puerto_salida_Q'),
 (0.004618403605946912, 'puerto_salida_C'),
 (0.0, 'genero_hombre')]

In [27]:
#%%

variables_independientes.iloc[0]

pasajero_nvo = pd.DataFrame(
    {'clase_billete': [3.],
     'edad': [22.],
     'n_hermanos_esposos': [1.],
     'n_hijos_padres': [0.],
     'precio_billete': [7.25],
     'genero_hombre': [1.],
     'genero_mujer': [0.],
     'puerto_salida_C': [0.],
     'puerto_salida_Q': [0.],
     'puerto_salida_S': [1.]
     }
    )

arbol.predict(pasajero_nvo)

#%%

array([0], dtype=int64)

In [28]:

'''
Éstos son los parámetros más importantes para los modelos DecisionTreeClassifier 
de sklearn:

criterion : El criterio para calcular la reducción de impureza (ganancia de información) 
al hacer una partición. Se puede elegir entre gini, o entropy

max_depth : La profundidad máxima del árbol. Definimos profundidad como el número 
de nodos que atraviesa una observación (cuantas "preguntas" se le hacen).

max_features: El máximo numero de particiones potenciales que se consideran al 
evaluar un nodo.

max_leaf_nodes : Límite de hojas para el árbol.

min_impurity_decrease : la ganancia de información mínima en un nodo para hacer 
una partición. (Si no hay ninguna partición que cumpla este criterio, se para el 
                desarrollo del árbol en dicho nodo).

class_weight : Para clases imbalanceadas, podemos pasar el argumento class_weight, 
como un diccionario de {clase: peso} para que sklearn tenga en cuenta los pesos. 
Alternativamente, podemos pasar el string balanced para que sklearn genere pesos 
en función del número de muestras de cada clase.
'''

'''
Control del máximo de profundidad
'''

arbol_simple = tree.DecisionTreeClassifier(max_depth=3)
arbol_simple.fit(variables_independientes, variable_objetivo)


DecisionTreeClassifier(max_depth=3)

In [29]:


dibujar_arbol(arbol_simple)


In [30]:
cross_val_score(arbol_simple,
                variables_independientes, 
                variable_objetivo, 
                scoring="roc_auc", 
                cv=40).mean()

0.8480444902319902

In [31]:

for k in range(3,21):
    arbol_simple = tree.DecisionTreeClassifier(max_depth=k)
    arbol_simple.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)
    calificacion = cross_val_score(arbol_simple, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()
    print(f"{(k,calificacion)}")



(3, 0.8502134227428346)
(4, 0.8608540587952354)
(5, 0.8438866253572137)
(6, 0.8282831395184337)
(7, 0.8342171009818069)
(8, 0.8158888888888889)
(9, 0.8126814927991399)
(10, 0.807311829782418)
(11, 0.7986367314602608)
(12, 0.7873871770930594)
(13, 0.7709972271736978)
(14, 0.7785628837393543)
(15, 0.771696347225759)
(16, 0.7728170783464902)
(17, 0.7791547123900064)
(18, 0.7749465240641711)
(19, 0.7713755764932235)
(20, 0.7738375350140055)


In [33]:
#%%
'''
balanceo
'''

arbol_balanceado = tree.DecisionTreeClassifier(max_depth=4, class_weight="balanced")
arbol_balanceado.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)
dibujar_arbol(arbol_balanceado)

cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()



0.8507758254817078

In [34]:
#%%

'''
Además del algoritmo CART para generar árboles, scikit-learn también proporciona 
una clase de arboles llamada ExtraTreeClassifier, o Extremely Random Trees 
(Árboles Extremadamente Aleatorios). En estos árboles, en lugar de seleccionar 
en cada nodo la párticion que proporciona la mayor ganancia de información, 
¡se decide una partición al azar!.
'''

arbol_aleatorio = tree.ExtraTreeClassifier(max_features=1)
arbol_aleatorio.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

dibujar_arbol(arbol_aleatorio)


In [35]:

cross_val_score(arbol_aleatorio, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc",
                cv=10).mean()

0.760124635712871